In [1]:
import pandas as pd
import itertools
import collections
import re

In [2]:
import os
import sys
import nltk
import pandas as pd
import json
import ast
import itertools
import numpy as np
from sklearn.preprocessing import normalize
import nltk
from nltk import bigrams
from pandas.io.json import json_normalize

In [3]:
df = pd.read_csv(r"C:\Users\Rasta\Desktop\data.csv")

In [4]:
df

,Name,Attributes,"boudaries(X,y,Width,Height)",labels_with_attributes,labels
0,0.jpg,"{'boot': {'pred_score': 0.9877846240997314, 'a...","{'leather,black boot': [132.0513153076172, 453...","['leather black boot', 'dark black sunglasses'...","dict_keys(['boot', 'sunglasses', 'coat', 'bag'..."
1,1.jpg,"{'suit': {'pred_score': 0.8407937288284302, 'a...","{'gray,blue suit': [64.69796752929688, 152.610...","['gray blue suit', 'burgundy red maroon shirt'...","dict_keys(['suit', 'shirt', 'sandal', 'sunglas..."
2,2.jpg,"{'dress': {'pred_score': 0.9954791069030762, '...","{'tan,brown dress': [249.31253051757812, 334.2...","['tan brown dress', 'red bag', 'red heel', 'st...","dict_keys(['dress', 'bag', 'heel', 'jacket'])"
3,3.jpg,"{'dress': {'pred_score': 0.952522337436676, 'a...","{'blue,green,white dress': [282.48492431640625...","['blue green white dress', 'teal light blue bl...","dict_keys(['dress', 'skirt', 'shoe', 'sandal',..."
4,4.jpg,"{'dress': {'pred_score': 0.986148476600647, 'a...","{'hanging,blue dress': [69.56578063964844, 47....","['hanging blue dress', 'tan brown sandal', 'wh...","dict_keys(['dress', 'sandal', 'bag', 'shirt', ..."
...,...,...,...,...,...
33370,33370.jpg,"{'hat': {'pred_score': 0.8971937298774719, 'at...","{'red,blue hat': [242.73876953125, 65.00129699...","['red blue hat', 'black and white white black ...","dict_keys(['hat', 'shoe', 'shirt', 'sweater', ..."
33371,33371.jpg,{'necklace': {'pred_score': 0.9878942370414734...,"{'gold necklace': [339.7559509277344, 226.5207...","['gold necklace', 'white dress', 'black skirt'...","dict_keys(['necklace', 'dress', 'skirt', 'hand..."
33372,33372.jpg,"{'shirt': {'pred_score': 0.8176797032356262, '...","{'white,blue shirt': [117.45890045166016, 170....","['white blue shirt', 'black and white white bl...","dict_keys(['shirt', 'sneaker', 'pant', 'tights..."
33373,33373.jpg,"{'dress': {'pred_score': 0.9423413872718811, '...","{'floral,white dress': [244.40399169921875, 14...","['floral white dress', 'gold tan brown heel', ...","dict_keys(['dress', 'heel', 'shirt', 'handbag'..."


In [5]:
i = 0

print(df['labels_with_attributes'][i])
print(eval(df['boudaries(X,y,Width,Height)'][i]).keys())

['leather black boot', 'dark black sunglasses', 'black coat', 'black bag', 'black glasses', 'black jacket', 'black shirt', 'black purse', 'black handbag', 'blue black pant']
dict_keys(['leather,black boot', 'dark,black sunglasses', 'black coat', 'gray,black coat', 'black bag', 'black glasses', 'black jacket', 'black boot', 'black shirt', 'black,gray shirt', 'leather,black bag', 'black purse', 'gray,large,black purse', 'black handbag', 'blue,black pant'])


In [6]:
eval(df['Attributes'][0]).keys()

dict_keys(['boot', 'sunglasses', 'coat', 'bag', 'glasses', 'jacket', 'shirt', 'purse', 'handbag', 'pant'])

In [7]:
def extract_from_sample(dic):
    a = [dic[k]['attrs'] for k in dic.keys()]

    occur = [] 
    for i,obj in enumerate(a):
        sent =  ' '.join([d[0] for d in obj] ) + ' ' + list(dic.keys())[i]
        occur.append(sent)
    
    return occur

In [8]:
bad = pd.read_csv('bad_pairs.csv')
bad.drop(columns=["Unnamed: 0"],inplace=True)
bad

,0,1
0,glasses,sunglasses
1,bag,purse
2,pant,trousers
3,coat,jacket
4,shoe,sandal
5,shoe,heel
6,heel,sandal
7,hoodie,jacket
8,shirt,dress
9,shirt,sweater


In [9]:
dic = bad.to_dict('split')['data']
dic

[['glasses', 'sunglasses'],
 ['bag', 'purse'],
 ['pant', 'trousers'],
 ['coat', 'jacket'],
 ['shoe', 'sandal'],
 ['shoe', 'heel'],
 ['heel', 'sandal'],
 ['hoodie', 'jacket'],
 ['shirt', 'dress'],
 ['shirt', 'sweater'],
 ['handbag', 'purse'],
 ['bag', 'handbag'],
 ['shoe', 'boot'],
 ['pant', 'skirt'],
 ['shirt', 'blouse'],
 ['backpack', 'bag'],
 ['shirt', 'clothes'],
 ['boot', 'heel'],
 ['shoe', 'sneaker'],
 ['sandal', 'boot'],
 ['shirt', 'shirt'],
 ['pant', 'tights'],
 ['pant', 'dress'],
 ['trousers', 'tights'],
 ['suit', 'pant'],
 ['boot', 'boot'],
 ['shoe', 'shoe'],
 ['goggles', 'sunglasses'],
 ['goggles', 'glasses'],
 ['dress', 'suit'],
 ['dress', 'robe'],
 ['shirt', 'tshirt'],
 ['dress', 'dress'],
 ['dress', 'outfit'],
 ['dress', 'costume'],
 ['sneaker', 'boot'],
 ['hat', 'hat'],
 ['coat', 'clothes'],
 ['jacket', 'clothes'],
 ['sweater', 'jersey'],
 ['scarf', 'dress'],
 ['bag', 'bag'],
 ['heel', 'heel'],
 ['purse', 'purse'],
 ['jacket', 'vest'],
 ['uniform', 'jersey'],
 ['skirt', '

In [10]:
def manual_clean(x):
    di = eval(x).copy()
    a = eval(x).keys()
    b = eval(x).copy()
    
    
    for el1 in a:
        for el2 in a:
            if el2!=el1:
                if ([el1,el2] in dic) :
                    if el1 in di.keys():
                        del di[el1]

    return di   

In [11]:
a = manual_clean(df['Attributes'][0])
a.keys()

dict_keys(['boot', 'sunglasses', 'jacket', 'shirt', 'purse', 'pant'])

In [12]:
from tqdm import tqdm

tqdm.pandas()

df['Attributes'] = df['Attributes'].progress_apply(lambda x: manual_clean(x))

100%|██████████████████████████████████████████████████████████████████████████| 33375/33375 [00:13<00:00, 2562.58it/s]


In [13]:
def extract_from_dic(dic):
    a = [dic[k]['attrs'] for k in dic.keys()]

    occur = [] 
    for i,obj in enumerate(a):
        sent =  ' '.join([d[0] for d in obj] ) + ' ' + list(dic.keys())[i]
        occur.append(sent)
    
    return occur

df['labels_with_attributes'] = df['Attributes'].apply(lambda x: extract_from_dic(x))

df['labels'] = df['Attributes'].apply(lambda x: x.keys())

In [14]:
def filter_boundary(x):
    bound = eval(x["boudaries(X,y,Width,Height)"])
    final = bound.copy()
    attr = x["labels_with_attributes"]
    
    for el in bound.keys():
        if not el.replace(",", " ") in attr:
            del final[el]
    return final

def rowFunc(row):
    return filter_boundary(row)

def rowIndex(row):
    return row.name

df['d'] = df.apply(rowFunc, axis=1)
df['rowIndex'] = df.apply(rowIndex, axis=1)

df['boudaries(X,y,Width,Height)'] = df['d']
df.drop(columns=['d','rowIndex'],inplace= True) 


In [15]:
df

,Name,Attributes,"boudaries(X,y,Width,Height)",labels_with_attributes,labels
0,0.jpg,"{'boot': {'pred_score': 0.9877846240997314, 'a...","{'leather,black boot': [132.0513153076172, 453...","[leather black boot, dark black sunglasses, bl...","(boot, sunglasses, jacket, shirt, purse, pant)"
1,1.jpg,"{'suit': {'pred_score': 0.8407937288284302, 'a...","{'gray,blue suit': [64.69796752929688, 152.610...","[gray blue suit, burgundy red maroon shirt, bl...","(suit, shirt, sandal, sunglasses, jacket, trou..."
2,2.jpg,"{'dress': {'pred_score': 0.9954791069030762, '...","{'tan,brown dress': [249.31253051757812, 334.2...","[tan brown dress, red bag, red heel, standing ...","(dress, bag, heel, jacket)"
3,3.jpg,"{'dress': {'pred_score': 0.952522337436676, 'a...","{'blue,green,white dress': [282.48492431640625...","[blue green white dress, tan brown sandal, blu...","(dress, sandal, jacket)"
4,4.jpg,"{'dress': {'pred_score': 0.986148476600647, 'a...","{'hanging,blue dress': [69.56578063964844, 47....","[hanging blue dress, tan brown sandal, white b...","(dress, sandal, bag, jacket)"
...,...,...,...,...,...
33370,33370.jpg,"{'hat': {'pred_score': 0.8971937298774719, 'at...","{'red,blue hat': [242.73876953125, 65.00129699...","[red blue hat, black and white white black sho...","(hat, shoe, sweater, bag)"
33371,33371.jpg,{'necklace': {'pred_score': 0.9878942370414734...,"{'gold necklace': [339.7559509277344, 226.5207...","[gold necklace, white dress, leather brown han...","(necklace, dress, handbag, sweater, heel, sung..."
33372,33372.jpg,{'sneaker': {'pred_score': 0.43405601382255554...,"{'black and white,white,black sneaker': [418.9...","[black and white white black sneaker, red tigh...","(sneaker, tights, hat, dress, glasses)"
33373,33373.jpg,"{'dress': {'pred_score': 0.9423413872718811, '...","{'floral,white dress': [244.40399169921875, 14...","[floral white dress, gold tan brown heel, beig...","(dress, heel, handbag)"


In [16]:
docToken = df['labels_with_attributes']

In [17]:
docToken[3]

['blue green white dress', 'tan brown sandal', 'blue jacket']

In [18]:
from tqdm import tqdm

check_s = []

for i in tqdm(docToken):
    bigrams = [(a,b) for a in i for b in i if a!=b ]
    check_s = check_s + bigrams 

100%|███████████████████████████████████████████████████████████████████████████| 33375/33375 [01:47<00:00, 310.19it/s]


In [19]:
check_s[0:100]

[('leather black boot', 'dark black sunglasses'),
 ('leather black boot', 'black jacket'),
 ('leather black boot', 'black shirt'),
 ('leather black boot', 'black purse'),
 ('leather black boot', 'blue black pant'),
 ('dark black sunglasses', 'leather black boot'),
 ('dark black sunglasses', 'black jacket'),
 ('dark black sunglasses', 'black shirt'),
 ('dark black sunglasses', 'black purse'),
 ('dark black sunglasses', 'blue black pant'),
 ('black jacket', 'leather black boot'),
 ('black jacket', 'dark black sunglasses'),
 ('black jacket', 'black shirt'),
 ('black jacket', 'black purse'),
 ('black jacket', 'blue black pant'),
 ('black shirt', 'leather black boot'),
 ('black shirt', 'dark black sunglasses'),
 ('black shirt', 'black jacket'),
 ('black shirt', 'black purse'),
 ('black shirt', 'blue black pant'),
 ('black purse', 'leather black boot'),
 ('black purse', 'dark black sunglasses'),
 ('black purse', 'black jacket'),
 ('black purse', 'black shirt'),
 ('black purse', 'blue black p

In [20]:
from collections import Counter
import operator

dic = Counter(check_s)

data = dict( sorted(dic.items(), key=operator.itemgetter(1),reverse=True))

In [22]:
cleaned = {}
keys = []

for d in tqdm(data.keys()) :
    if not (d[1],d[0]) in keys:
        cleaned[d] = data[d]
        keys.append(d)

100%|█████████████████████████████████████████████████████████████████████████| 296990/296990 [22:33<00:00, 219.43it/s]


In [23]:
data

{('black dress', 'black heel'): 441,
 ('black heel', 'black dress'): 441,
 ('black boot', 'black dress'): 267,
 ('black dress', 'black boot'): 267,
 ('black heel', 'black jacket'): 243,
 ('black jacket', 'black heel'): 243,
 ('black boot', 'black jacket'): 213,
 ('black jacket', 'black boot'): 213,
 ('black hat', 'black boot'): 186,
 ('black boot', 'black hat'): 186,
 ('black dress', 'black jacket'): 168,
 ('black jacket', 'black dress'): 168,
 ('black boot', 'black trousers'): 162,
 ('black trousers', 'black boot'): 162,
 ('white black dress', 'black heel'): 156,
 ('black heel', 'white black dress'): 156,
 ('black boot', 'black bag'): 152,
 ('black bag', 'black boot'): 152,
 ('dark black sunglasses', 'black boot'): 152,
 ('black boot', 'dark black sunglasses'): 152,
 ('black heel', 'black hat'): 151,
 ('black hat', 'black heel'): 151,
 ('dark black sunglasses', 'black purse'): 150,
 ('black purse', 'dark black sunglasses'): 150,
 ('black heel', 'black purse'): 150,
 ('black purse', 'b

In [24]:
len(cleaned)

148495

In [25]:
len(data)

296990

In [26]:
bigram_df = pd.DataFrame.from_dict(cleaned,  orient='index')

bigram_df= bigram_df.reset_index()

bigram_df.columns = ['bigram', 'count']

bigram_df

,bigram,count
0,"(black dress, black heel)",441
1,"(black boot, black dress)",267
2,"(black heel, black jacket)",243
3,"(black boot, black jacket)",213
4,"(black hat, black boot)",186
...,...,...
148490,"(floral white dress, beige tan brown handbag)",1
148491,"(gold tan brown heel, beige tan brown handbag)",1
148492,"(white blue boot, red dress)",1
148493,"(white blue boot, brown purse)",1


In [27]:
bigram_df[0:20]

,bigram,count
0,"(black dress, black heel)",441
1,"(black boot, black dress)",267
2,"(black heel, black jacket)",243
3,"(black boot, black jacket)",213
4,"(black hat, black boot)",186
5,"(black dress, black jacket)",168
6,"(black boot, black trousers)",162
7,"(white black dress, black heel)",156
8,"(black boot, black bag)",152
9,"(dark black sunglasses, black boot)",152


In [28]:
bigram_df.to_csv("Final_co-occurence_polyvore_Adel.csv",index=False)